Learning how to build a chatbox. Following Tech with Time (https://www.youtube.com/watch?v=wypVcNIH6D4&list=PLzMcBGfZo4-ndH9FoC4YWHGXG5RZekt-Q)

In [16]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random
import json

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Masood\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


The intent file is formated as a JSON file. Here is the format:
{"intents": [
        {"tag": "greating",
        "patterns": ["Hi"],
        "response" ["Hello"],
        "context_set": ""
        }
        
The basic structor is as follows:
    - person types a message
    - the chatbot, through deeplearning, tags the message
    - the chatbot then returns with one of the responses

In [11]:
# reading our json file
with open ('intents.json') as file:
    data = json.load(file)
    
#print(data)  # just to make sure we correctly loaded/read our file
#print(data['intents'])

In [21]:
# save each patter/grouping into a variable 
words = []
labels = []
docs_x = [] # for each patter, we need to know what intent is
docs_y = [] 

for intent in data['intents']:
    for pattern in intent['patterns']:
        # stemming will break each word in our pattern and break it down into the root word
        # ex. "Is anyone there?": the root is "there" ignoreing the other words and ?
        # we need to tokkenize (getting all the words)
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(pattern)
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
            labels.append(intent['tag'])
            

# stem all words and remove duplicates 
words = [stemmer.stem(w.lower()) for w in words]
words = sorted(list(set(words))) # set removes duplicates, list converts set back to list, sorted just sorts the words

labels = sorted(labels)


To run data in a neurl network, you need to convert the string data into numbers. To do this, we'll convert our strings into "bag of words" (one-hot encoding)

In [22]:
training = []
output = []

out_empty = [0 for _ in range(len(label))]

for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w) for w in doc]
    
    # one-hot encoding
    for w in words:
        if w in wrds: 
            bag.append(1)
        else:
            bag.append(0)
            
    out_row = out_empty[:]
    out_row[labels.index(docs_y[x])] - 1

    training.append(bag)
    bag.append(out_row)


training = np.array(training)
output = np.array(output)

NameError: name 'classes' is not defined